<a href="https://colab.research.google.com/github/danielDelfi/FinancialEngineering_IR_xVA/blob/main/10y_T_Note_Futures_Pricing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Pricing function for 10y Treasury futures

# First, price a generic 10y Treasury Note with an arbitrary coupon (as long as the yield is below 6%) but with only 6.6 years to maturity.
# Due to the quality option, assume that the short futures holder can deliver a T-note with the lowest allowed time to maturity, i.e. 6.6 years.

from QuantLib import *

# set the date on which bond and futures are priced
pricingDate = Date(8, 2, 2023) # futures is priced on Feb 8, 2023
Settings.instance().evaluationDate = pricingDate

# create a yield curve to be used for pricing bond and futures  
# yields are constant-maturity and coupon-bearing, but here assumed to be spot rates
#"2023-02-08 00:00:00": {"10y": 3.63, "1m": 4.64, "1y": 4.87, "20y": 3.86, "2y": 4.45, "30y": 3.7, "3m": 4.72, "3y": 4.08, "5y": 3.82, "6m": 4.88, "7y": 3.75}

spotDates = [Date(8,2,2023),
            Date(8,3,2023), Date(8,5,2023), Date(8,8,2023), Date(8,2,2024), Date(10,2,2025), Date(9,2,2026),
            Date(8,2,2028), Date(8,2,2030), Date(8,2,2033), Date(10,2,2053)] # skipping 20y yield to have a smoother curve

spotRates = [0.00,
            0.0464, 0.0472, 0.0488, 0.0487, 0.0445, 0.0408, 0.0382, 0.0375, 0.0363, 0.0370] # skipping 20y yield # pulled from Fed's yield curve data

dayCount = ActualActual(ActualActual.Bond) # this counts days for the purpose of calculating accrued interest (dirty bond price) using the bond market convention
calendar = UnitedStates(UnitedStates.GovernmentBond) # adjusts for the holidays and non-trading days in the Treasury market
interpolation = Linear() # use linear interpolation to get yields for all maturities  
compounding = Compounded # turn on compounding
compoundingFrequency = Semiannual # semiannual rule used by the Treasury market
spotCurve = ZeroCurve(spotDates, spotRates, dayCount, calendar, 
                      interpolation, compounding, compoundingFrequency) # this function simply collects yields and interpolates feeded yields 
                      # (and adjusts for calendar and compounding); it does not create a zero-coupon curve

spotCurveHandle = YieldTermStructureHandle(spotCurve) # yield curve object passed into a pricing function below

# spotCurve.nodes() # uncomment to check the rates input in the ZeroCurve object 

ModuleNotFoundError: ignored

In [ ]:
# Construct cash flow schedule
issueDate = Date(18, 2, 2020) # date Treasury was issued
maturityDate = Date(15, 2, 2030) # date Treasury matures
period = Period(Semiannual) # Treasury coupons are paid every six months  
calendar = UnitedStates(UnitedStates.GovernmentBond) # calendar for the US Treasury market
businessConvention = Following # this is to ensure that coupons are paid on business days
dateGeneration = DateGeneration.Backward # generate dates for coupon payments according to the calendar and business days
monthEnd = False # technical, requires that dates be end of month

schedule = Schedule(issueDate, maturityDate, period, calendar, 
                    businessConvention, businessConvention, dateGeneration, monthEnd) # sets up cash flow schedule

couponRate = [0.01625] # this is supposed to be an arbitrary coupon rate than can be adjusted for by the conversion factor below
settlementDays = 0 # technical, adjusts for mismatch between trade and settlement
faceValue = 100 # price quoted per $100 par or face value

fixedRateBond = FixedRateBond(settlementDays, faceValue, schedule, couponRate, dayCount)
                # sets up the bond object to be passed into pricing function below

NameError: ignored

In [ ]:
# price the bond using the constructed yield curve
bondEngine = DiscountingBondEngine(spotCurveHandle)
fixedRateBond.setPricingEngine(bondEngine)

#print(fixedRateBond.NPV())
print('Clean Price:', fixedRateBond.cleanPrice())
print('Dirty Price:', fixedRateBond.dirtyPrice())
print('Accrued Interest:', fixedRateBond.accruedAmount())
#print('Day Counting:', fixedRateBond.dayCounter())
print('Pricing as of:', pricingDate)

Clean Price: 86.98198683382589
Dirty Price: 87.76357650773893
Accrued Interest: 0.7815896739130411
Pricing as of: February 8th, 2023


In [ ]:
# As stated above, assume that the priced bond with 6y6m left to maturity is cheapest-to-deliver and the
# futures contract matures in March 2023 
# (T-Note futures have four maturity months: March, June, September, and December)
futuresSettlementDate = Date(22,3,2023) # settlement date for the March 23 futures 
strike = 100 # par value
ctdBond = fixedRateBond # price of the underlying 10y bond
ctdConversionFactor = 0.7674 # this is the conversion factor for a bond with 6y6m left to maturity and a coupon of 1.625%
            # converstion factors are published by CME at 
            # https://www.cmegroup.com/trading/interest-rates/us-treasury-futures-conversion-factor-lookup-tables.html

futures = FixedRateBondForward(pricingDate, futuresSettlementDate, 
                                  Position.Long, strike, settlementDays,
                                  dayCount, calendar, businessConvention,
                                  ctdBond, spotCurveHandle, spotCurveHandle)

futuresPrice = futures.cleanForwardPrice()/ctdConversionFactor
print('Model Futures Price:',futuresPrice)
print('Market Futures Price:',113.25)

Model Futures Price: 113.5354892861363
Market Futures Price: 113.25
